In [1]:
from ml.features.preprocessing import get_data
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from ml.ml_functions.registry.model_registry import log_sklearn_model_to_mlflow
from ml.database.influxdb_manager import InfluxDBOperations
from ml.ml_functions.serve.mlflow_model_serve import mlflow_model_prediction
import mlflow

## Load Dataset with new Features

In [2]:
stock_data, last_day_df = get_data(save_data=False)

[*********************100%%**********************]  1 of 1 completed


/home/erik/DataspellProjects/stock-prediction/ml/features/preprocessing.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"SMA {time_period}"] = ta.SMA(inputs, timeperiod=time_period)
/home/erik/DataspellProjects/stock-prediction/ml/features/preprocessing.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"EMA {time_period}"] = ta.EMA(inputs, timeperiod=time_period)
/home/erik/DataspellProjects/stock-prediction/ml/features/preprocessing.py:54: SettingWithCopyWarning: 
A value is trying to be se

In [3]:
stock_data

,Close,Volume,Rendite,SMA 10,EMA 10,EMA 20,WMA 10,Momentum 10,SAR,RSI,...,ADOSC,%K,%D,+DMI,-DMI,ADX,up_band,mid_band,low_band,Target
Date,,,,,,,,,,,,,,,,,,,,,
1980-02-20,-1.033702,-0.990544,1.407289,-1.033750,-1.034365,-1.035603,-1.033867,-0.040226,-1.030372,0.697028,...,-0.369802,0.667696,0.279467,-0.211528,-1.125428,3.814284,-1.034267,-1.034972,-1.034718,0
1980-02-21,-1.034772,-0.986759,-0.938515,-1.033790,-1.034495,-1.035572,-1.034093,-0.081172,-1.030607,0.225440,...,-0.372643,-0.936635,-0.183188,-0.242706,-1.297858,3.569440,-1.034316,-1.034889,-1.034494,0
1980-02-22,-1.034987,-0.988507,-0.219311,-1.033901,-1.034641,-1.035565,-1.034351,-0.095730,-1.030832,0.133727,...,-0.373097,-0.062536,-0.196569,-0.473517,-1.093509,3.267324,-1.034363,-1.034829,-1.034319,0
1980-02-25,-1.036525,-0.993282,-1.349343,-1.034317,-1.035040,-1.035706,-1.034869,-0.157240,-1.031048,-0.470359,...,-0.375632,-0.741007,-1.033066,-0.644275,-0.941199,2.930565,-1.034336,-1.034842,-1.034374,0
1980-02-26,-1.035941,-0.992829,0.471777,-1.034600,-1.035260,-1.035778,-1.035204,-0.130306,-1.031399,-0.253478,...,-0.373702,0.396575,-0.241397,-0.807557,-0.967238,2.595932,-1.034309,-1.034881,-1.034484,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-10,2.831342,0.915533,1.345299,2.749055,2.765737,2.751196,2.781552,5.347513,2.650640,0.662350,...,1.598938,1.226363,0.183734,0.450536,-0.390639,-0.117401,2.759173,2.724713,2.685265,1
2023-11-13,2.828024,0.737157,-0.109157,2.771098,2.778319,2.759854,2.797121,4.380462,2.675728,0.618070,...,1.615126,0.250117,0.008181,0.375143,-0.492632,-0.155045,2.764821,2.726424,2.682771,1
2023-11-14,2.903684,1.460534,1.651053,2.798291,2.802395,2.774921,2.822457,5.420856,2.701957,1.185186,...,1.825897,0.566533,1.213419,1.342921,-0.812384,-0.049406,2.786466,2.731951,2.671083,1


In [4]:
X = stock_data.drop(['Target'], axis=1)
train_x, test_x, train_y, test_y = train_test_split(X, stock_data['Target'], test_size=0.15, random_state=42)

## Train your Sklearn-Model

In [5]:
from ml.ml_functions.training.train_extra_tree import train_extra_tree

model, accuracy = train_extra_tree(stock_data)

Modellname: ExtraTreesClassifier
Genauigkeit: 0.8477341389728097


## Model Registry 

### mlflow server --host 0.0.0.0 --port 5000


In [7]:
actual_model_path = log_sklearn_model_to_mlflow(model=model, accuracy=accuracy)

2023/12/03 15:37:40 INFO mlflow.tracking.fluent: Experiment with name 'sp500_prediction' does not exist. Creating a new experiment.
2023/12/03 15:37:44 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: best_ExtraTreesClassifier_model, version 1


## Save current feature data for prediction

In [17]:
stock_data, last_day_df = get_data(save_data=True, new_model=(etc.__class__.__name__, accuracy))

[*********************100%%**********************]  1 of 1 completed


/home/erik/DataspellProjects/stock-prediction/ml/features/preprocessing.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"SMA {time_period}"] = ta.SMA(inputs, timeperiod=time_period)
/home/erik/DataspellProjects/stock-prediction/ml/features/preprocessing.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"EMA {time_period}"] = ta.EMA(inputs, timeperiod=time_period)
/home/erik/DataspellProjects/stock-prediction/ml/features/preprocessing.py:54: SettingWithCopyWarning: 
A value is trying to be se

Complete. Return to the InfluxDB UI.
Saved Data to Influx and CSV


### Predict the new Data with Mlflow-Model

In [18]:
result = mlflow_model_prediction(model=actual_model_path)

NameError: name 'mlflow_model_prediction' is not defined

In [ ]:
result